This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

---

In [2]:
%cd /code

/code


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

---

# Tools and more

Before showning an example using a `Chain`, it will be helpful to understand the various tools that can be used in a chain.

## DuckDuckGo Search

We can use the DuckDuckGo search engine to retrieve the top URLs associated with a search query. The `DuckDuckGoSearch` object is a callable object that returns a list of dictionaries. Each item in the list corresponds to a search result.

In [5]:
from llm_chain.tools import DuckDuckGoSearch

duckduckgo_search = DuckDuckGoSearch(top_n=3)
duckduckgo_search("What is a large language model?")

[{'title': 'What are LLMs, and how are they used in generative AI?',
  'href': 'https://www.computerworld.com/article/3697649/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html',
  'body': "Large language models are the algorithmic basis for chatbots like OpenAI's ChatGPT and Google's Bard. The technology is tied back to billions — even trillions — of parameters that can make them..."},
 {'title': 'What is a large language model and how does it work? - Fast Company',
  'href': 'https://www.fastcompany.com/90884581/what-is-a-large-language-model',
  'body': 'Large language models are the foundational technology behind recent artificial intelligence advancements like ChatGPT.'},
 {'title': 'What are Large Language Models - MachineLearningMastery.com',
  'href': 'https://machinelearningmastery.com/what-are-large-language-models/',
  'body': 'Large language models (LLMs) are recent advances in deep learning models to work on human languages. Some great use case of L

---

# Chains

**A `Chain` consists of individual links. Each link can be thought of as a task in a workflow (e.g. document search, web search, or chat model). Each link is a callable (either a function or a callable object) where the output of one link is the input of the next link. So a `Chain` is a simple mechanism that takes an input and sends the input to the first link, and the propegates the output of the first link to the second link, until the end of the chain is reached, and returns the final result.** 

A `Chain` object aggregates the history (messages, usage, etc) of all of the links. More specifically, it aggregiates all of the lists of `Record` objects for any link that has a `history` property, such as the OpenAIChat class. The `Chain` class provides various history properties (e.g. `history`, `usage_history`, `message_history`) depending on the type of `Record` returned by each link's history property (e.g. the `usage_history` property returns all of the `UsageRecord` objects across any link whose `history` property returns `UsageRecord`s).

## Chat model w/ web search 

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   link/task   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: `What is the meaning of life?`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one link matches the input of the next link.**



Let's define the chain and corresponding objects in the next cell:

- the `ChromaDocumentIndex` is an object that has two main methods: `add()` to add Document objects to the search index and `search()` to search for Documents based on a provided Document. It's a callable object that when called with a list of Documents, adds the Documents to the index, and when called with a single Document, searches the index and returns a list of similar Documents.
- the `DocSearchTemplate` is a callable object that is initialized with a document index and, when called (with a `str` input representing the original question/prompt), looks up the most similar/relevant documents based on the query, and returns a new prompt with the original question and with the most relevant chunks.
-  

### Initial objects in chain

In [5]:
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on question
duckduckgo_search = DuckDuckGoSearch(top_n=3)
# Embeddings model used for document index/search
embeddings_model = OpenAIEmbeddings(model_name='text-embedding-ada-002')
# Store and lookup most relevant documents
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate looks up most relevant documents based on query
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|'),
)

def search_results_to_docs(results: list[dict]) -> list[Document]:
    """For each url (`href` in `results`), extracts text, cleans, returns Document."""
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]

# A `Value` object is a simple caching mechanism. It is a callable object that, 
# when called with a value passed in, caches the value in an instance field and returns the value;
# and when called without a value passed in returns the cached value stored in the instance field.
# In this case, it's a way for us to pass our question to the `DuckDuckGoSearch` object and then
# also pass it to the `DocSearchTemplate` object in the middle of the chain.
initial_question = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

In [6]:
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    initial_question,
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    initial_question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `chain()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, but should also match the response returned
response = chain("What is the meaning of life?")

TypeError: ChromaDocumentIndex.search() got an unexpected keyword argument 'n_results'

In [ ]:
response

In [ ]:
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

duckduckgo_search = DuckDuckGoSearch(top_n=3)

document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'))
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')

streaming_callback = lambda x: print(x.response, end='|')
streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo', streaming_callback=streaming_callback)

# for each url, extracts text, cleans, returns doc
def search_results_to_docs(results: list[dict]) -> list[Document]:
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]

initial_question = Value()
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    initial_question,
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    initial_question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])

response = chain("What is the meaning of life?")